In [14]:
import re
import os
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [15]:
root_path = 'crime_results/results_dt=0.1/'
A_pattern = 'A_(\d+).pkl'
rho_pattern = 'rho_(\d+).pkl'
skip = 1
x_pts = np.arange(0, 512, 1.0, np.float)

In [16]:
A_files = [f for f in os.listdir(root_path) if re.match(A_pattern, f)]
rho_files = [f for f in os.listdir(root_path) if re.match(rho_pattern, f)]

In [17]:
def get_number(pattern, f):
    return int(re.match(pattern, f).groups()[0])

In [18]:
n_A_files = max(get_number(A_pattern, f) for f in A_files)
n_rho_files = max(get_number(rho_pattern, f) for f in rho_files)

assert n_A_files == n_rho_files
n = n_A_files + 1

sorted_A = sorted(A_files, key=lambda f: get_number(A_pattern,f))
sorted_rho = sorted(rho_files, key=lambda f: get_number(rho_pattern,f))

In [19]:
def get_values(file_name):
    with open(os.path.join(root_path, file_name), 'rb') as f:
        return pickle.load(f)

In [20]:
As = []
rhos = []
for A_file, rho_file in zip(sorted_A, sorted_rho):
    A = get_values(A_file)
    rho = get_values(rho_file)
    
    if np.any(np.isnan(np.concatenate(A))) or np.any(np.isnan(np.concatenate(rho))):
        continue
    
    As.append(A[::skip])
    rhos.append(rho[::skip])

In [21]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)
from matplotlib import animation
animation.Animation._repr_html_ = anim_to_html

In [22]:
for i in range(len(As[0])):
    plt.figure(figsize=(10, 10))
    plt.xlim(x_pts.min(), x_pts.max())
    plt.ylim(0,1)
    for ix in range(len(As)):
        plt.plot(x_pts, As[ix][i], c='green', alpha=0.2, label='A')
        plt.plot(x_pts, rhos[ix][i], c='blue', alpha=0.2, label='rho')
    A_mean, = plt.plot(x_pts, np.column_stack([A[i] for A in As]).mean(axis=1), label='A', c='green')
    rho_mean, = plt.plot(x_pts, np.column_stack([rho[i] for rho in rhos]).mean(axis=1), label='rho', c='blue')
    plt.legend(['A', 'rho'], handles=[A_mean, rho_mean])
    plt.savefig('crime_results/figures/frame_{}.png'.format(i))
    plt.close()

In [31]:
# First set up the figure, the axis, and the plot element we want to animate
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_xlim(x_pts.min(), x_pts.max())
ax.set_ylim(0,1)
fig = plt.gcf()
A_lines = [ax.plot([], [], label='A', c='green', alpha=0.2)[0] for _ in xrange(len(As))]
rho_lines = [ax.plot([], [], label='rho', c='blue', alpha=0.2)[0] for _ in xrange(len(As))]

A_mean, = ax.plot([], [], label='A', c='green')
rho_mean, = ax.plot([], [], label='rho', c='blue')
plt.legend(['A', 'rho'], handles=[A_mean, rho_mean])
# initialization function: plot the background of each frame
def init():
    return []

# animation function.  This is called sequentially
def animate(i):
    for ix in xrange(len(As)):
        try:
            A_lines[ix].set_data(x_pts, As[ix][i])
            rho_lines[ix].set_data(x_pts, rhos[ix][i])
        except:
            print ix, i
            raise
    A_mean.set_data(x_pts, np.column_stack([A[i] for A in As]).mean(axis=1))
    rho_mean.set_data(x_pts, np.column_stack([rho[i] for rho in rhos]).mean(axis=1))
    return A_lines + rho_lines + [A_mean, rho_mean]

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(As[0]), interval=20, blit=True)
plt.close()
anim

In [32]:
anim.save('sim_dt=0.01.mp4', fps=20, extra_args=['-vcodec', 'libx264']);